In [ ]:
# Correr para reiniciar Kernel y la sesion.
!kill -9 -1

# Proyecto Final (Texto)

### Curso Deep Learning y Redes Neuronales - MIIA4406_01

#### Universidad de los Andes

Integrantes:

|Nombre|Codigo|
|---------------------|---------------------|
|Saby Espinel|*201215868*|
|Diego Salazar|*201628925*| <br>
<br>
______________________________________________________________________________________________________________________

## Librerias

In [1]:
# Verificar que se esta usando GPU
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3660442571289135848, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 185794560
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15056358130697963547
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [3]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.metrics import r2_score, roc_auc_score
#from sklearn.model_selection import train_test_split
#from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from skimage import io

# Text processing
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split

# Keras NN
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, Embedding, LSTM
from keras.optimizers import RMSprop
from keras.callbacks import History
from keras.models import Model
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

%matplotlib inline

Using TensorFlow backend.


In [4]:
##### Para usar en CoLab #####
!pip install -U -q PyDrive
!pip install textblob
!pip install tqdm
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files


    100% |████████████████████████████████| 634kB 5.1MB/s 


In [ ]:
##### Para usar en CoLab #####
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# >>> Read Data CoLab

In [ ]:
##### Para CoLab #####
# Verificacion de contenido de carpetas en el Drive con su respectivo ID
#file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
file_list = drive.ListFile({'q': "'1gX_xAhoYwIwmOVRd33lAuf4pCM3cChB6' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: TextImgDeepLearningColab.ipynb, id: 1sEAr0qGqVwDkmZJjw4o1UC3owfqykJma
title: pred_genres_img_RF.csv, id: 15rsRbXoE0U40VgtVdLduYe5Ie4tmlCbC
title: Final Project.pptx, id: 17-MkUf4ltoGnoNHS8XhuI2PzG0Igd9YF
title: ~$Final Project.pptx, id: 1ppcxsvHrCXQwQqCNtyQW0UvHyhdDqNb4
title: ExampleImgRF.ipynb, id: 1Pg2-wYIUr866PzKvUy8iXsstZT87uE6x
title: Resultados_Jupyter, id: 1ANK5CbRmFQ6w-jbSzkkUO_bdjnUyXB6I
title: Resultados_CoLab, id: 17Sk_UrlcK0BQIPAZ018Gjp8JxQZmm38p
title: TextImgDeepLearning.ipynb, id: 1Cqw5bxyGIg6Pm-8hCAFuBh8-T-lwu_oi
title: images_resize_gray, id: 1AuAOku0WZor-SegS9YWlg0gjqjlguogj
title: images_resize_gray.zip, id: 1mcEsaLxakpcQqYZl1pe_qtIEh4FXj0Wx
title: images_original, id: 16FnNSHtTQzQBgLbXKs77btkYJrWIH5IF
title: images_original.zip, id: 1_Grjc2zdV2gHA7unPg_I7qTEdM6zl1Ok
title: ExampleTextRF.ipynb, id: 1awW3xWv99qbvcMP1hWTIJjmdmKq8Ku42
title: Proyecto 2_birds_DL (1).ipynb, id: 1mRpiAc1wbDI226Zw56g6NvPHK0ebcACn
title: images.zip, id: 1NMIKcqCHJIDaQi4M9mbjrZf0qlpXt

In [ ]:
##### Para CoLab #####
trainDrive = drive.CreateFile({'id': '1SKNGe6FFghASG38LjtUaoTrmozUibRCZ'})
trainDrive.GetContentFile('dataTraining.csv')
dataTraining=pd.read_csv("dataTraining.csv", index_col=0)

testDrive = drive.CreateFile({'id': '1JWWmvheWnofZl2zqY-7Gu2rJprceYB80'})
testDrive.GetContentFile('dataTesting.csv')
dataTesting=pd.read_csv("dataTesting.csv", index_col=0)

# >>> Read Data Jupyter

In [ ]:
##### Para CoLab #####
# Lectura de archivos en Jupyter desde el PC
path = 'C:/Users/da.salazarb/Google Drive (dasalazarb@unal.edu.co)/AppliedDeepLearningClass-master/finalProject'
dataTraining = pd.read_csv(os.path.join(path, 'dataTraining.csv'), encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv(os.path.join(path, 'dataTesting.csv'), encoding='UTF-8', index_col=0)
#print(dataTraining.head())
#print(dataTesting.head())

## Cleaning data (dataTraining['post'] and dataTesting['post'])

In [ ]:
# Definicion de funcion para preprocesar el texto
def preProssText(df, commonWord, numberCommonWords, rareWord, numberRareWords, spellCorrect):
  # Remove punctutation
  df['plot'] = df['plot'].str.replace('[^\w\s]','')
  
  # Lower case
  df['plot'] = df['plot'].apply(lambda x: " ".join(x.lower() for x in x.split()))
  
  # Remove Common words
  if commonWord == True:
    freq = pd.Series(' '.join(df['plot']).split()).value_counts()[:numberCommonWords]
    df['plot'] = df['plot'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
  
  # Remove Rare words
  if rareWord == True:
    freq = pd.Series(' '.join(df['plot']).split()).value_counts()[-numberRareWords:]
    df['plot'] = df['plot'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
  
  # Spelling correction -> Tener cuidado puede cambiar los valores. Ej: "ur" por "or" en vez de "your"
  if spellCorrect == True:
    from textblob import TextBlob
    df['plot'] = df['plot'].apply(lambda x: str(TextBlob(x).correct()))
    
  return df

In [ ]:
# Pre-procesamiento de texto
dataTraining = preProssText(dataTraining, commonWord=True, numberCommonWords=10, rareWord=True, numberRareWords=10, spellCorrect=False)
dataTesting = preProssText(dataTesting, commonWord=True, numberCommonWords=10, rareWord=True, numberRareWords=10, spellCorrect=False)

In [ ]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most story single father who takes eight year ...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,serial killer decides teach secrets satisfying...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,sweden female blackmailer with disfiguring fac...,"['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,friday afternoon new york president tredway co...,['Drama'],7.4
2582,1990,Narrow Margin,los angeles editor publishing house carol hunn...,"['Action', 'Crime', 'Thriller']",6.6


In [ ]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,who meets by fate shall be sealed by fate ther...
4,1978,Midnight Express,true story billy hayes an american college stu...
5,1996,Primal Fear,martin vail left chicago da office become succ...
6,1950,Crisis,husband wife americans dr eugene mrs helen fer...
7,1959,The Tingler,coroner scientist dr warren chapin researching...


***************************************************************************************************

# Create count vectorizer with ngrams

In [7]:
# define a function that accepts text and returns a list of lemmas
import nltk
nltk.download('wordnet')

def split_into_lemmas(text):
    from nltk.stem import WordNetLemmatizer
    wordnet_lemmatizer = WordNetLemmatizer()
    
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word) for word in words]

[nltk_data] Downloading package wordnet to /content/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# define a function that accepts text and returns a list of stem
def split_into_stem(text):
    stemmer = SnowballStemmer('english')
    text = text.lower()
    words = text.split()
    return [stemmer.stem(word) for word in words]

In [9]:
vect = CountVectorizer(analyzer=split_into_lemmas, ngram_range=(1, 2), max_features=1000, stop_words='english')
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 1000)

In [10]:
print(vect.get_feature_names()[:50])

['!', '"', '$', '&', "'", '(', ')', ',', '-', '.', '/', ':', ';', '?', 'a', 'able', 'about', 'accepts', 'accident', 'accidentally', 'across', 'act', 'action', 'actor', 'actress', 'actually', 'adam', 'adult', 'adventure', 'affair', 'after', 'again', 'against', 'age', 'agent', 'ago', 'agrees', 'air', 'alex', 'alice', 'alien', 'alive', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always']


***************************************************************************************************

# Create y

In [11]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [ ]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [13]:
y_genres.shape

(7895, 24)

******************************************************************************************************************************************

# Split train and test

In [ ]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.1, random_state=42)

# Train DeepNeuralNetwork

In [ ]:
# Modelo
top_words = 1000
max_review_length = 1000
# Set the model
def textModel(max_review_length):
    model = Sequential()
    model.add(Embedding(top_words, 32, input_length=max_review_length))
    #model.add(BatchNormalization())
    model.add(LSTM(256, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(24, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=['accuracy'])
    return model

In [ ]:
ix_train=np.random.choice(X_train.shape[0], 1000)
ix_test=np.random.choice(X_test.shape[0], 100)
model=textModel(max_review_length)
model.fit(X_train[ix_train], y_train_genres[ix_train], epochs=3, batch_size=50, validation_data=(X_test[ix_test], y_test_genres[ix_test]))
#model.fit(X_dtm, y_genres, epochs=3, batch_size=10)
# epochs=5, batch_size=5 -> acc: 0.18

Train on 1000 samples, validate on 100 samples
Epoch 1/3
1000/1000 [==============================] - 95s 95ms/step - loss: 8.9106 - acc: 0.0730 - val_loss: 9.0495 - val_acc: 0.0000e+00
Epoch 2/3
1000/1000 [==============================] - 94s 94ms/step - loss: 8.7159 - acc: 0.0980 - val_loss: 9.9464 - val_acc: 0.0500
Epoch 3/3
1000/1000 [==============================] - 95s 95ms/step - loss: 8.5696 - acc: 0.1040 - val_loss: 25.1097 - val_acc: 0.0500


Alternativa 1:


In [ ]:
# Modelo
top_words = 1000
max_review_length = 1000
# Set the model
def textModel(max_review_length):
    model = Sequential()
    model.add(Embedding(top_words, 32, input_length=max_review_length))
    #model.add(BatchNormalization())
    model.add(LSTM(256))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(24, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=['accuracy'])
    return model

In [17]:
ix_train=np.random.choice(X_train.shape[0], 1000)
ix_test=np.random.choice(X_test.shape[0], 100)
model=textModel(max_review_length)
model.fit(X_train[ix_train], y_train_genres[ix_train], epochs=3, batch_size=50, validation_data=(X_test[ix_test], y_test_genres[ix_test]))
#acc: 0.258

Train on 1000 samples, validate on 100 samples
Epoch 1/3


ResourceExhaustedError: ignored

Alternativa 2:

# Prediction on dataTesting dataset

In [ ]:
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = model.predict_proba(X_test_dtm)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_genres_text_RF.csv', index_label='ID')

In [ ]:
files.download('pred_genres_text_RF.csv')

******************************************************************************************************************************************

******************************************************************************************************************************************